In [ ]:
!git clone https://github.com/google-research/timesfm.git
!cd timesfm && pip install .[torch]

In [ ]:
! pip install utilsforecast

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from utilsforecast.plotting import plot_series
from utilsforecast.evaluation import evaluate
from utilsforecast.losses import *

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import torch
import timesfm

In [ ]:
DATA_URL = "https://raw.githubusercontent.com/marcopeix/FoundationModelsForTimeSeriesForecasting/refs/heads/main/data/Walmart.csv"

df = pd.read_csv(DATA_URL)
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')

df.head()

## Zero-shot forecasting

In [ ]:
HORIZON = 13

In [ ]:
model = timesfm.TimesFM_2p5_200M_torch.from_pretrained("google/timesfm-2.5-200m-pytorch")

model.compile(
    timesfm.ForecastConfig(
        max_context=1024,
        max_horizon=256,
        normalize_inputs=True,
        use_continuous_quantile_head=True,
        force_flip_invariance=True,
        infer_is_positive=True,
        fix_quantile_crossing=True,
    )
)

In [ ]:
unique_stores = sorted(df['Store'].unique())
inputs_list = []

for store_id in unique_stores:
    store_df = df[df['Store'] == store_id].sort_values(by='Date')
    inputs_list.append(store_df['Weekly_Sales'].values)

point_forecast, quantile_forecast = model.forecast(
    horizon=HORIZON,
    inputs=inputs_list
)

In [ ]:
quantile_forecast.shape

In [ ]:
def create_forecast_df(
    quantile_forecast_array: np.array,
    original_df: pd.DataFrame,
    id_col: str,
    time_col: str,
    target_col: str,
    horizon: int,
    freq: str,
):

    num_series, forecast_horizon, num_quantiles_output = quantile_forecast_array.shape

    all_forecast_rows = []
    unique_ids = sorted(original_df[id_col].unique())


    for i, id in enumerate(unique_ids):
        # Get the last known date for this store from the original DataFrame
        id_hist_df = original_df[original_df[id_col] == store_id].sort_values(by=time_col)
        last_known_date = id_hist_df[time_col].iloc[-1]

        # Generate forecast dates
        forecast_dates = pd.date_range(start=last_known_date, periods=horizon + 1, freq=freq)[1:]

        for h in range(horizon):
            forecast_row = {
                id_col: id,
                time_col: forecast_dates[h],
                'timesfm': quantile_forecast_array[i, h, 5],       # Median
                'timesfm-lo-80': quantile_forecast_array[i, h, 1], # 10th percentile
                'timesfm-hi-80': quantile_forecast_array[i, h, 9]  # 90th percentile
            }
            all_forecast_rows.append(forecast_row)

    return pd.DataFrame(all_forecast_rows)

In [ ]:
fcsts_df = create_forecast_df(
    quantile_forecast_array=quantile_forecast,
    original_df=df,
    id_col='Store',
    time_col='Date',
    target_col='Weekly_Sales',
    horizon=HORIZON,
    freq="W-FRI",
)
fcsts_df.head()

In [ ]:
plot_series(
    df=df,
    forecasts_df=fcsts_df,
    id_col="Store",
    time_col="Date",
    target_col="Weekly_Sales",
    level=[80],
    max_ids=6,
)

## Cross-validation

In [ ]:
def timesfm_cv(
    df: pd.DataFrame,
    model: timesfm.timesfm_2p5.timesfm_2p5_torch.TimesFM_2p5_200M_torch,
    horizon: int,
    n_windows: int,
    id_col: str,
    time_col: str,
    target_col: str,
    freq: str,
):
    all_cv_forecasts = []

    max_date = df[time_col].max()

    for i in range(n_windows):
        # Calculate the cutoff date for the current window
        cutoff_date = max_date - pd.Timedelta((n_windows - i) * horizon, unit=freq[0])

        # Create a training DataFrame up to the cutoff_date
        df_train = df[df[time_col] <= cutoff_date]

        # Prepare inputs_list
        unique_ids = sorted(df_train[id_col].unique())
        inputs_list = []
        for id in unique_ids:
            sub_df = df_train[df_train[id_col] == id].sort_values(by=time_col)
            inputs_list.append(sub_df[target_col].values)

        # Generate forecasts
        _, quantile_forecast = model.forecast(
            horizon=horizon,
            inputs=inputs_list
        )

        # Convert forecasts to DataFrame
        fcsts_df = create_forecast_df(
            quantile_forecast_array=quantile_forecast,
            original_df=df_train,
            id_col=id_col,
            time_col=time_col,
            target_col=target_col,
            horizon=horizon,
            freq=freq,
        )

        # Add cutoff column
        fcsts_df['cutoff'] = cutoff_date

        all_cv_forecasts.append(fcsts_df)

    cv_df = pd.concat(all_cv_forecasts, ignore_index=True)
    cv_df = cv_df.merge(df[[id_col, time_col, target_col]], how="left", on=[id_col, time_col])
    return cv_df

In [ ]:
cv_df = timesfm_cv(
    df=df,
    model=model,
    horizon=HORIZON,
    n_windows=3,
    id_col='Store',
    time_col='Date',
    target_col='Weekly_Sales',
    freq="W-FRI",
)
cv_df.head()

In [ ]:
plot_series(
    df=df,
    forecasts_df=cv_df.drop(columns=["cutoff", "Weekly_Sales"]),
    id_col="Store",
    time_col="Date",
    target_col="Weekly_Sales",
    level=[80],
    max_ids=6,
)

In [ ]:
eval_df = evaluate(
    cv_df.drop(columns=["cutoff"]),
    metrics=[mae, smape],
    models=['timesfm'],
    target_col='Weekly_Sales',
    id_col='Store',
    time_col="Date",
    agg_fn="mean"
)
eval_df

## Forecasting with covariates

In [ ]:
!pip install "jax[cuda]" scikit-learn

In [ ]:
model = timesfm.TimesFM_2p5_200M_torch.from_pretrained("google/timesfm-2.5-200m-pytorch")

model.compile(
    timesfm.ForecastConfig(
        max_context=1024,
        max_horizon=256,
        normalize_inputs=True,
        use_continuous_quantile_head=True,
        force_flip_invariance=True,
        infer_is_positive=True,
        fix_quantile_crossing=True,
        return_backcast=True,
    )
)

In [ ]:
def timesfm_cv(
    df: pd.DataFrame,
    model: timesfm.timesfm_2p5.timesfm_2p5_torch.TimesFM_2p5_200M_torch,
    horizon: int,
    n_windows: int,
    id_col: str,
    time_col: str,
    target_col: str,
    freq: str,
    dynamic_categorical_cols: list[str] | None = None,
):
    all_cv_forecasts = []

    max_date = df[time_col].max()

    for i in range(n_windows):
        # Calculate the cutoff date for the current window
        cutoff_date = max_date - pd.Timedelta((n_windows - i) * horizon, unit=freq[0])

        # Create a training DataFrame up to the cutoff_date
        df_train = df[df[time_col] <= cutoff_date]

        # Prepare inputs_list
        unique_ids = sorted(df_train[id_col].unique())
        inputs_list = []
        for id in unique_ids:
            sub_df = df_train[df_train[id_col] == id].sort_values(by=time_col)
            inputs_list.append(sub_df[target_col].values)

        # Prepare dynamic categorical covariates if specified
        dynamic_categorical_covariates = None
        if dynamic_categorical_cols:
            dynamic_categorical_covariates = {}

            for col_name in dynamic_categorical_cols:
                covariate_values = []

                for id in unique_ids:
                    # Get the full time series for this ID (train + horizon)
                    sub_df_full = df[df[id_col] == id].sort_values(by=time_col)

                    # Filter to get: historical data up to cutoff + horizon periods after cutoff
                    forecast_end_date = cutoff_date + pd.Timedelta(horizon, unit=freq[0])
                    sub_df_with_horizon = sub_df_full[
                        sub_df_full[time_col] <= forecast_end_date
                    ]

                    # Extract covariate values (input_size + horizon)
                    covariate_seq = sub_df_with_horizon[col_name].tolist()
                    covariate_values.append(covariate_seq)

                dynamic_categorical_covariates[col_name] = covariate_values

        # Generate forecasts
        if dynamic_categorical_covariates:
            _, quantile_forecast_list = model.forecast_with_covariates(
                inputs=inputs_list,
                dynamic_categorical_covariates=dynamic_categorical_covariates,
            )
            # Stack the list of arrays into a single array
            quantile_forecast = np.stack(quantile_forecast_list, axis=0)
        else:
            _, quantile_forecast = model.forecast(
                horizon=horizon,
                inputs=inputs_list
            )

        # Convert forecasts to DataFrame
        fcsts_df = create_forecast_df(
            quantile_forecast_array=quantile_forecast,
            original_df=df_train,
            id_col=id_col,
            time_col=time_col,
            target_col=target_col,
            horizon=horizon,
            freq=freq,
        )

        # Add cutoff column
        fcsts_df['cutoff'] = cutoff_date

        all_cv_forecasts.append(fcsts_df)

    cv_df = pd.concat(all_cv_forecasts, ignore_index=True)
    cv_df = cv_df.merge(df[[id_col, time_col, target_col]], how="left", on=[id_col, time_col])
    return cv_df

In [ ]:
cv_df_exog = timesfm_cv(
    df=df,
    model=model,
    horizon=HORIZON,
    n_windows=3,
    id_col='Store',
    time_col='Date',
    target_col='Weekly_Sales',
    freq="W-FRI",
    dynamic_categorical_cols=['Holiday_Flag'],
)
cv_df_exog.head()

In [ ]:
plot_series(
    df=df,
    forecasts_df=cv_df_exog.drop(columns=["cutoff", "Weekly_Sales"]),
    id_col="Store",
    time_col="Date",
    target_col="Weekly_Sales",
    level=[80],
    max_ids=6,
)

In [ ]:
exog_eval_df = evaluate(
    cv_df_exog.drop(columns=["cutoff"]),
    metrics=[mae, smape],
    models=['timesfm'],
    target_col='Weekly_Sales',
    id_col='Store',
    time_col="Date",
    agg_fn="mean"
)
exog_eval_df